Data from https://openrepair.org/open-data/downloads/

(Source: https://openrepair.org/news/open-repair-alliance-dataset-increases-to-81000-records-of-community-repair/)

In [ ]:
import polars as pl

pl.Config.set_fmt_str_lengths(100)

In [ ]:
from kedro.config import OmegaConfigLoader
from kedro.io import DataCatalog

conf_loader = OmegaConfigLoader(
    ".", base_env=".", default_run_env=".", custom_resolvers={
        "pl": lambda obj: getattr(pl, obj),
    }
)
conf_catalog = conf_loader.get("catalog")
catalog = DataCatalog.from_config(conf_catalog)

In [ ]:
categories = catalog.load("openrepair-0_3-categories")
categories.head()

In [ ]:
df = catalog.load("openrepair-0_3-events-raw")
df.head()

In [ ]:
len(df)

In [ ]:
df = df.select(pl.all().exclude("product_category")).join(
    categories, on="product_category_id"
)
df.head()

In [ ]:
len(df)

In [ ]:
df["country"].value_counts().sort("counts", descending=True).head()

In [ ]:
df["product_category"].value_counts().sort(by="counts", descending=True).head(10)

Minor correction needed for repair barrier: one part of [the standard](https://standard.openrepair.org/standard.html) specifies `"Item too worn out"`, the other says `"Product too worn out"`, but they are the same.

In [ ]:
df["repair_barrier_if_end_of_life"].value_counts()

In [ ]:
df = df.with_columns(
    pl.col("repair_barrier_if_end_of_life").map_dict(
        {"Item too worn out": "Product too worn out"},
        default=pl.col("repair_barrier_if_end_of_life"),
    )
)
df["repair_barrier_if_end_of_life"].value_counts()